In [2]:
pip install islp

Note: you may need to restart the kernel to use updated packages.


In [17]:
import statsmodels.api as sm

In [18]:
 from statsmodels.stats.outliers_influence \
import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm

In [19]:
 from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize,
poly)

In [45]:
import pandas as pd
import numpy as np

In [4]:
pip list

Package                       Version
----------------------------- ---------------
aiobotocore                   2.4.2
aiofiles                      22.1.0
aiohttp                       3.8.3
aioitertools                  0.7.1
aiosignal                     1.2.0
aiosqlite                     0.18.0
alabaster                     0.7.12
anaconda-anon-usage           0.4.3
anaconda-catalogs             0.2.0
anaconda-client               1.12.0
anaconda-cloud-auth           0.1.4
anaconda-navigator            2.5.1
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
async-timeout                 4.0.2
atomicwrites                  1.4.0
attrs                         22.1.0
autograd                      1.6.2
autograd-

In [5]:
from ISLP import load_data
Publication = load_data('Publication')
Publication.columns

Index(['posres', 'multi', 'clinend', 'mech', 'sampsize', 'budget', 'impact',
       'time', 'status'],
      dtype='object')

In [7]:
Publication.head

<bound method NDFrame.head of      posres  multi  clinend      mech  sampsize     budget  impact       time  \
0         0      0        1       R01     39876   8.016941  44.016  11.203285   
1         0      0        1       R01     39876   8.016941  23.494  15.178645   
2         0      0        1       R01      8171   7.612606   8.391  24.410678   
3         0      0        1  Contract     24335  11.771928  15.402   2.595483   
4         0      0        1  Contract     33357  76.517537  16.783   8.607803   
..      ...    ...      ...       ...       ...        ...     ...        ...   
239       0      0        0       R01      4105   2.703653   5.355  65.018480   
240       1      0        0       R44       181   1.117084   0.000  66.989733   
241       0      0        0       K23       104   0.472321   0.000   9.987680   
242       0      0        0       R21        69   0.404710   0.000  21.979466   
243       1      0        0       R01      1699   2.957751   0.000   4.632444  

In [14]:
#Filter the dataset to only clinical trials that have been published.

clinical = Publication[Publication['status'] == 1]
clinical.head

<bound method NDFrame.head of      posres  multi  clinend      mech  sampsize     budget  impact       time  \
0         0      0        1       R01     39876   8.016941  44.016  11.203285   
1         0      0        1       R01     39876   8.016941  23.494  15.178645   
2         0      0        1       R01      8171   7.612606   8.391  24.410678   
3         0      0        1  Contract     24335  11.771928  15.402   2.595483   
4         0      0        1  Contract     33357  76.517537  16.783   8.607803   
..      ...    ...      ...       ...       ...        ...     ...        ...   
226       1      0        1       U54        31   5.649036  38.278   7.983573   
232       1      0        0       R44      1800   0.988677   3.299  35.022587   
237       1      0        0       R01       247   1.305481   0.000   9.691992   
238       1      0        0       R01       247   1.305481   0.000  16.459959   
239       0      0        0       R01      4105   2.703653   5.355  65.018480  

In [37]:
#Start by fitting a linear regression that models impact as a function of all other variables.
X = Publication.drop("impact",axis=1)#specifies 1 is a column 0 is row
X = sm.add_constant(X)
print(X.dtypes)






const            float64
posres             int64
multi              int64
clinend            int64
sampsize           int64
budget           float64
time             float64
status             int64
mech_K01           uint8
mech_K23           uint8
mech_P01           uint8
mech_P50           uint8
mech_R01           uint8
mech_R18           uint8
mech_R21           uint8
mech_R24, K24      uint8
mech_R42           uint8
mech_R44           uint8
mech_RC2           uint8
mech_U01           uint8
mech_U54           uint8
dtype: object


In [36]:

X = pd.get_dummies(X, columns=['mech'], drop_first=True)
print(X.columns)

KeyError: "None of [Index(['mech'], dtype='object')] are in the [columns]"

converted column mech to dummy variables using one hot encoding to get rid of the category data type in pandas

In [31]:
y = Publication['impact']
print(y.dtypes)

float64


In [38]:
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 impact   R-squared:                       0.635
Model:                            OLS   Adj. R-squared:                  0.602
Method:                 Least Squares   F-statistic:                     19.36
Date:                Sun, 11 Feb 2024   Prob (F-statistic):           3.42e-38
Time:                        18:56:37   Log-Likelihood:                -903.64
No. Observations:                 244   AIC:                             1849.
Df Residuals:                     223   BIC:                             1923.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             5.4891      3.584      1.532

evaluate by p values
starting with dropping posres, sampsize, budget, mech columns due to high p values
Now to fit a new model with X less these values, and same y

In [42]:
columns_to_drop = ['posres', 'sampsize', 'budget']

columns_to_drop += [col for col in X.columns if col.startswith('mech')]
X_reduced = X.drop(columns=columns_to_drop)

print(X_reduced)


     const  multi  clinend       time  status
0      1.0      0        1  11.203285       1
1      1.0      0        1  15.178645       1
2      1.0      0        1  24.410678       1
3      1.0      0        1   2.595483       1
4      1.0      0        1   8.607803       1
..     ...    ...      ...        ...     ...
239    1.0      0        0  65.018480       1
240    1.0      0        0  66.989733       0
241    1.0      0        0   9.987680       0
242    1.0      0        0  21.979466       0
243    1.0      0        0   4.632444       0

[244 rows x 5 columns]


looks clean, lets fit a new model to see if its better

In [43]:
modelReduced = sm.OLS(y, X_reduced).fit()
print(modelReduced.summary())

                            OLS Regression Results                            
Dep. Variable:                 impact   R-squared:                       0.612
Model:                            OLS   Adj. R-squared:                  0.605
Method:                 Least Squares   F-statistic:                     94.21
Date:                Sun, 11 Feb 2024   Prob (F-statistic):           5.60e-48
Time:                        19:25:41   Log-Likelihood:                -910.98
No. Observations:                 244   AIC:                             1832.
Df Residuals:                     239   BIC:                             1849.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4788      1.461      1.697      0.0

In [48]:
correlation_matrix = X_reduced.corr()
print(correlation_matrix)

         const     multi   clinend      time    status
const      NaN       NaN       NaN       NaN       NaN
multi      NaN  1.000000  0.476594 -0.216176  0.293741
clinend    NaN  0.476594  1.000000 -0.291213  0.313143
time       NaN -0.216176 -0.291213  1.000000 -0.188822
status     NaN  0.293741  0.313143 -0.188822  1.000000


The reduced model is arguably better. It maintains a high level of explanatory power with significantly fewer predictors, leading to a simpler model that is easier to interpret and likely more generalizable. The lower AIC and BIC values also support the reduced model as a more optimal balance between fit and complexity.

take a look at the correlation between your variables. Do you see any issues? Which variables are most highly correlated? Investigate and describe the relationship between the two sets of highest correlated variables.



the variables multi and clinend have the highest correlation coefficient among the pairs of independent variables, with a value of 0.476594. This indicates a moderate positive correlation between these two variables. 

Fit another linear regression model that includes all variables except for one in each pair of most highly correlated variables. How well does this model fit the data? How does this compare to your earlier models?

In [54]:
# here, since I identified multi and clinend have the highest correlation coefficients,
#I am going to drop multi and fit a new model


X_correlated = X_reduced.drop(columns = 'multi')
print(X_correlated)

     const  clinend       time  status
0      1.0        1  11.203285       1
1      1.0        1  15.178645       1
2      1.0        1  24.410678       1
3      1.0        1   2.595483       1
4      1.0        1   8.607803       1
..     ...      ...        ...     ...
239    1.0        0  65.018480       1
240    1.0        0  66.989733       0
241    1.0        0   9.987680       0
242    1.0        0  21.979466       0
243    1.0        0   4.632444       0

[244 rows x 4 columns]


Model without multi,,,,

In [55]:
modelCorrelated = sm.OLS(y, X_correlated).fit()
print(modelCorrelated.summary())

                            OLS Regression Results                            
Dep. Variable:                 impact   R-squared:                       0.558
Model:                            OLS   Adj. R-squared:                  0.553
Method:                 Least Squares   F-statistic:                     101.0
Date:                Tue, 13 Feb 2024   Prob (F-statistic):           2.56e-42
Time:                        13:02:58   Log-Likelihood:                -926.82
No. Observations:                 244   AIC:                             1862.
Df Residuals:                     240   BIC:                             1876.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9271      1.553      1.885      0.0


After removing the multi variable from the regression model, the updated analysis shows a decrease in the R-squared value from 0.612 to 0.558 and a slight decrease in the adjusted R-squared from 0.605 to 0.553. This indicates that the model now explains less of the variability in the dependent variable, impact, compared to the original model. Despite this reduction, the F-statistic remains high, and the model is statistically significant, as evidenced by a Prob (F-statistic) of 2.56e-42. The coefficients for the remaining variables (clinend, time, and status) are statistically significant, with clinend showing an increased coefficient, suggesting a stronger relationship with impact in the absence of multi